<a href="https://colab.research.google.com/github/agatagruza/private-ai/blob/master/SPAIC_Project13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 13: Securing Federated Learning

##Trusted Aggregator

In [0]:
pip install syft

In [2]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)
from torch import nn, optim

W0727 04:53:34.421897 140692690282368 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0727 04:53:34.439057 140692690282368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# creatring virtual workers
bob = sy.VirtualWorker(hook, id = "bob") # data owner
alice = sy.VirtualWorker(hook, id = "alice") # data owner
# secure worker is a trusted 3rd party that is able to perform good aggregation
secure_worker = sy.VirtualWorker(hook, id = "secure_worker")

In [0]:
# Inrom worker, that other workers exsist. 
# Those are references to the other workers. 
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([bob, alice])

In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True) 
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True) 

In [0]:
# get pointers to training data on each worker by
# sending some trainig data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [0]:
alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [9]:
print(bobs_data)
print(bobs_target)
print(alices_data)
print(alices_target)

(Wrapper)>[PointerTensor | me:76321689297 -> bob:56465019317]
(Wrapper)>[PointerTensor | me:94723447321 -> bob:70744552807]
(Wrapper)>[PointerTensor | me:30858905216 -> alice:50573715008]
(Wrapper)>[PointerTensor | me:98341343148 -> alice:53350962861]


In [10]:
# Initialize a Toy Model
model = nn.Linear(2, 1)
print(model)

Linear(in_features=2, out_features=1, bias=True)


In [0]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

In [14]:
print(bobs_model)
print(alices_model)

Linear(in_features=2, out_features=1, bias=True)
Linear(in_features=2, out_features=1, bias=True)


In [0]:
# Two optimizers. One for the parameters Bob's model and one for the parametrs of alice's
bobs_optimizer = optim.SGD(params=bobs_model.parameters(), lr = 0.1)
alices_optimizer = optim.SGD(params=alices_model.parameters(), lr = 0.1)

In [17]:
print(bobs_optimizer)
print(alices_optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [19]:
for round in range(10):

    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_optim = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_optim = optim.SGD(params=alices_model.parameters(), lr=0.1)

    for i in range(10):
        # Bobs model in train
        bobs_optim.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target) ** 2).sum()
        bobs_loss.backward()

        bobs_optim.step()
        bobs_loss = bobs_loss.get().data
        bobs_loss

        # Alices model in train
        alices_optim.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target) ** 2).sum()
        alices_loss.backward()

        alices_optim.step()
        alices_loss = alices_loss.get().data  
        alices_loss
        
    # It iterates through every parameter in alices model and calls .move() 
    # on that parameter inline.
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    # AVERAGING ALICE AND BOB TOGETEHR !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 
    with th.no_grad():
      model.weight.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
      model.bias.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
  
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(0.0007) Alice:tensor(0.0017)
Bob:tensor(0.0010) Alice:tensor(0.0004)
Bob:tensor(0.0009) Alice:tensor(0.0002)
Bob:tensor(0.0007) Alice:tensor(0.0001)
Bob:tensor(0.0005) Alice:tensor(9.7172e-05)
Bob:tensor(0.0004) Alice:tensor(7.0800e-05)
Bob:tensor(0.0003) Alice:tensor(5.2400e-05)
Bob:tensor(0.0002) Alice:tensor(3.9094e-05)
Bob:tensor(0.0002) Alice:tensor(2.9287e-05)
Bob:tensor(0.0001) Alice:tensor(2.1987e-05)


In [20]:
secure_worker._objects

{1897046304: Parameter containing:
 tensor([[ 0.7078, -0.0291]], requires_grad=True),
 6601264388: Parameter containing:
 tensor([[ 0.7803, -0.0222]], requires_grad=True),
 7859216227: Parameter containing:
 tensor([[0.7527, 0.0150]], requires_grad=True),
 8822466949: Parameter containing:
 tensor([0.0208], requires_grad=True),
 12481839604: Parameter containing:
 tensor([[0.8386, 0.0093]], requires_grad=True),
 17515926099: Parameter containing:
 tensor([0.0158], requires_grad=True),
 17976413954: Parameter containing:
 tensor([0.4040], requires_grad=True),
 18792544878: Parameter containing:
 tensor([[ 0.8347, -0.0167]], requires_grad=True),
 21688418056: Parameter containing:
 tensor([[0.5659, 0.0535]], requires_grad=True),
 23452364558: Parameter containing:
 tensor([[ 0.7466, -0.0254]], requires_grad=True),
 25192060727: Parameter containing:
 tensor([[0.7855, 0.0126]], requires_grad=True),
 34359186220: Parameter containing:
 tensor([0.0137], requires_grad=True),
 36171987881: Pa

In [21]:
secure_worker.clear_objects()

<VirtualWorker id:secure_worker #objects:0>

**And voila.We have Federated Learnign  where the gradient aggregation happens on a neutral third party.**

##Intro to Additive Secret Sharing

In [0]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

In [0]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

In [0]:
bob_x_share = 2
alice_x_share = 3

bob_y_share = 5
alice_y_share = 2

bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

In [0]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

In [0]:
(bob_x_share + alice_x_share) % Q

##Build Methods for Encrypted, Decrypted, and Add

In [0]:
import random

Q = 23740629843760239486723

def encrypt(x, n_share=3):
    
    shares = list()
    for i in range(n_share-1):
        shares.append(random.randint(0,Q))  
    shares.append(Q - (sum(shares) % Q) + x)
    return tuple(shares)

In [0]:
def decrypt(shares):
    return sum(shares) % Q

In [0]:
def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [0]:
x = encrypt(10)
print(x)
y = encrypt(20)
print(y)
z = add(x, y)
print(z)
decrypt(

##Intro to Fixed Precision Encoding

In [0]:
BASE=10
PRECISION=4

In [0]:
def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q) / BASE**PRECISION

In [0]:
encode(4.578)

In [0]:
encode(5.7)

In [0]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

##Secret Sharing Using PySyft

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.share(bob, alice, secure_worker)

In [0]:
bob._objects

In [0]:
alice._objects

In [0]:
y = x + x
y

In [0]:
y.get()

##Fixed Precision using PySyft

In [0]:
x = th.tensor([0.1,0.2,0.3])
x

In [0]:
x = x.fix_prec()

In [0]:
x

In [0]:
y = x + x
y

In [0]:
y = y.float_prec()
y

##Shared Fixed Precision

In [0]:
x = th.tensor([0.1, 0.2, 0.3])
x = x.fix_prec().share(bob, alice, secure_worker)

In [0]:
y = x + x
y

In [0]:
y.get().float_prec()